# 5章 文章の穴埋め

- 以下で実行するコードには確率的な処理が含まれていることがあり、コードの出力結果と本書に記載されている出力例が異なることがあります。

BERTは文章の一部を`[MASK]`トークンに変換したものを読み込み、`[MASK]`トークンに当てはまるトークンが何かを予測するかたちで事前学習を行っている。同様に、BERTは一部文章の穴埋めを行うことができる。

## 5.1 パッケージのインストール

文章の穴埋めには、`transformers.BertForMaskedLM`クラスを用いる。

In [1]:
# 5-2

import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

## 5.2 BERTを用いた文章の穴埋め

トークナイザと穴埋めモデルをロードする。<br>
ロードにはそれぞれのクラスの`from_pretrained()`メソッドを使用する。

In [2]:
# 5-3

model_name = "cl-tohoku/bert-base-japanese-whole-word-masking"
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<br>

モデルをGPUメモリ上で動作させるには、以下を実行しておく

In [8]:
# model = model.cuda()

<br>

今回は、
```
今日は[MASK]へ行く。
```
という文章の`[MASK]`トークンの穴埋めをする。まずは文章をトークン化して確認してみる。

In [3]:
# 5-4

text = "今日は[MASK]へ行く。"
tokens = tokenizer.tokenize(text)
print(tokens)

['今日', 'は', '[MASK]', 'へ', '行く', '。']


<br>

`transformers.BertForMaskedLM`に符号化した文章を渡す。

In [4]:
# 5-5
# 符号化した文章のみほしい、系列長を揃える必要がない
# のでtokenizer()関数は必要なし
input_ids = tokenizer.encode(text, return_tensors='pt')
# input_ids = input_ids.cuda()

# BERTに入力する
with torch.no_grad():
    output = model(input_ids=input_ids) # 可変長引数を用いない場合、引数を指定する
    scores = output.logits

In [5]:
scores.size()

torch.Size([1, 8, 32000])

出力`transformers.modeling_outputs.MaskedLMOutput`のクラス変数`output.logits`として、語彙に含まれる各トークンの分類スコアを表すテンソル`scores`を得られる。<br>
具体的には、トークン化した文章 + 文頭トークン`[CLS]` + 文末トークン`[SEP]`の8トークンに対し、`WordPiece`が持つ32000語彙のそれぞれに当てはまる確率を計算する。

モデルが穴埋めした結果を見ていく。

In [6]:
# 5-6
# [MASK]トークン(トークンID:4)の位置
mask_position = input_ids[0].tolist().index(4)

# スコアが最もいいトークンのID(インデックス)を取り出す
id_best = scores[0, mask_position].argmax().item()

# トークンIDをトークンに置き換える
token_best = tokenizer.convert_ids_to_tokens(id_best)

# [MASK]をトークンに置き換える
text.replace("[MASK]", token_best)

'今日は東京へ行く。'

In [48]:
id_best

391

<br>

モデルが推論したスコア上位のトークンを抜き出してみる。<br>
関数を作成する。

In [65]:
# 5-7

def predict_mask_topk(
    text: str,
    tokeniser: BertJapaneseTokenizer, 
    model: BertForMaskedLM,
    num_topk: int
) -> (list, torch.Tensor):
    input_ids = tokenizer.encode(text, return_tensors='pt')
    # input_ids = input_ids.cuda()
    with torch.no_grad():
        output = model(input_ids=input_ids)
        scores = output.logits

    mask_position = input_ids[0].tolist().index(4)
    # torch.Tensor.topk(k)でtorch.Tensorの各要素のうち引数に指定した最大数k個だけ抜き出す
    scores_topk = scores[0][mask_position].topk(num_topk)
    
    values_topk = scores_topk.values.numpy() # GPUメモリに渡している場合、scores_topk.values.cpu().numpy()
    ids_topk = scores_topk.indices
    tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk)
    text_topk = [text.replace('[MASK]', token, 1) for token in tokens_topk] # 最初のトークンのみ変換する

    return (text_topk, values_topk)

In [38]:
text_topk, _ = predict_mask_topk(text, tokenizer, model, 10)
print(*text_topk, sep="\n")

今日は東京へ行く。
今日はハワイへ行く。
今日は学校へ行く。
今日はニューヨークへ行く。
今日はどこへ行く。
今日は空港へ行く。
今日はアメリカへ行く。
今日は病院へ行く。
今日はそこへ行く。
今日はロンドンへ行く。


自然な文章が出力されているのがわかる。

## 5.3 貪欲法を用いた複数`[MASK]`の穴埋め

次に、複数`[MASK]`がある場合を考える。
```
今日は[MASK][MASK]へ行く。
```
という文章の場合、`[MASK]`の組み合わせは$32000^2$通りとなるため、貪欲法という手法を用いて計算量を近似的に減らす。<br>
貪欲法は、

1. 一番最初にある`[MASK]`を最も高いスコアのトークンで穴埋めする。
2. 穴埋め後の文章に対して次の`[MASK]`を穴埋めする。

という処理を繰り返す。これにより、計算量$O$を$O=32000^2$から$O=32000\cdot2$まで減らすことができた。

In [70]:
# 5-8

def greedy_prediction(
    text: str,
    tokenizer: BertJapaneseTokenizer,
    model: BertForMaskedLM
) -> str:
    for _ in range(text.count("[MASK]")):
        text, _ = predict_mask_topk(text, tokenizer, model, 1)
        text = text[0]
    return text

In [71]:
text = "今日は[MASK][MASK]へ行く。"
greedy_prediction(text, tokenizer, model)

'今日は、東京へ行く。'

自然な文章が出力されているのがわかる。

一方で、BERTはコレまでの$N$グラム法のような、文章を前から順に出力するといったタスクは苦手である。

In [66]:
# 5-9

text = "今日は[MASK][MASK][MASK][MASK][MASK]"
greedy_prediction(text, tokenizer, model)

'今日は社会社会的な地位'

これは、BERTが事前学習において、文章のうちごく少数のトークンをマスクし、まわりの文脈から元のトークンを置き換えていることによる。

## 5.4 ビームサーチによる画像の穴埋め

貪欲法では最初の`[MASK]`から順番に1トークンずつ最も高いスコアで置き換えるが、これは全トークンの合計スコアが最も高いことを保証しない。そこで、よりよい近似手法として「ビームサーチ」を用いる。<br>
ビームサーチは、まず1つ目の`[MASK]`について上位10のトークンを穴埋めした文章を出力する。次に、生成された10の文章に対し、2つ目の`[MASK]`について上位10のトークンを穴埋めした文章を出力する。これにより生成された100の文章に対し、スコアの合計値が高い10の文章を選択する。その後は選択された10の文章に対し、同様に穴埋めを行う。

ビームサーチは、以下のように実装する。

In [76]:
text = "今日は[MASK][MASK]へ行く。"

In [77]:
num_mask = text.count("[MASK]")
text = [text]
scores_topk = np.array([0])

In [78]:
num_mask

2

In [79]:
text

['今日は[MASK][MASK]へ行く。']

In [80]:
scores_topk

array([0])

In [83]:
text_lst, scores = predict_mask_topk(text[0], tokenizer, model, 10)

In [84]:
text_lst

['今日は、[MASK]へ行く。',
 '今日は再び[MASK]へ行く。',
 '今日はその[MASK]へ行く。',
 '今日はあの[MASK]へ行く。',
 '今日は同じ[MASK]へ行く。',
 '今日はお[MASK]へ行く。',
 '今日はこの[MASK]へ行く。',
 '今日は新しい[MASK]へ行く。',
 '今日はゲーム[MASK]へ行く。',
 '今日は東京[MASK]へ行く。']

In [ ]:
# 5-10
def beam_search(text, tokenizer, bert_mlm, num_topk):
    """
    ビームサーチで文章の穴埋めを行う。
    """
    num_mask = text.count('[MASK]')
    text_topk = [text]
    scores_topk = np.array([0])
    for _ in range(num_mask):
        # 現在得られている、それぞれの文章に対して、
        # 最初の[MASK]をスコアが上位のトークンで穴埋めする。
        text_candidates = [] # それぞれの文章を穴埋めした結果を追加する。
        score_candidates = [] # 穴埋めに使ったトークンのスコアを追加する。
        for text_mask, score in zip(text_topk, scores_topk):
            text_topk_inner, scores_topk_inner = predict_mask_topk(
                text_mask, tokenizer, bert_mlm, num_topk
            )
            text_candidates.extend(text_topk_inner)
            score_candidates.append( score + scores_topk_inner )

        # 穴埋めにより生成された文章の中から合計スコアの高いものを選ぶ。
        score_candidates = np.hstack(score_candidates)
        idx_list = score_candidates.argsort()[::-1][:num_topk]
        text_topk = [ text_candidates[idx] for idx in idx_list ]
        scores_topk = score_candidates[idx_list]

    return text_topk

text = "今日は[MASK][MASK]へ行く。"
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')

In [ ]:
# 5-11
text = '今日は[MASK][MASK][MASK][MASK][MASK]'
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep='\n')